In [2]:
import undetected_chromedriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import ddddocr
import sqlite3

from modules.chrome import Chrome

In [3]:
# 初始化
URL = "https://tip.railway.gov.tw/tra-tip-web/tip/tip001/tip121/query"

options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
chrome = Chrome(options=options)
chrome.set_window_size(1600, 1000)

chrome.get(URL)

sql = sqlite3.connect("sql/tra_stations.sqlite")
cursor = sql.cursor()

In [38]:
# 列出地區
cursor.execute("SELECT area FROM stations ORDER BY id")
areas_set = set()
areas = []
for area in map(lambda x: x[0], cursor.fetchall()):
    if area not in areas_set:
        areas.append(area)
    areas_set.add(area)
for i in range(len(areas)):
    print(f"{i + 1:2d}: {areas[i]}")

 1: 基隆市
 2: 新北市
 3: 臺北市
 4: 桃園市
 5: 新竹縣
 6: 新竹市
 7: 苗栗縣
 8: 臺中市
 9: 彰化縣
10: 南投縣
11: 雲林縣
12: 嘉義縣
13: 嘉義市
14: 臺南市
15: 高雄市
16: 屏東縣
17: 臺東縣
18: 花蓮縣
19: 宜蘭縣


In [73]:
# 列出車站
AREA = "新竹縣"

cursor.execute(f"SELECT id, name FROM stations WHERE area = '{AREA}'")
stations = list(map(lambda x: {"id": x[0], "name": x[1]} ,cursor.fetchall()))
for station in stations:
    print(f"{station['id']:04d}: {station['name']}")

1150: 北湖
1160: 湖口
1170: 新豐
1180: 竹北
1193: 竹中
1194: 六家
1201: 上員
1202: 榮華
1203: 竹東
1204: 橫山
1205: 九讚頭
1206: 合興
1207: 富貴
1208: 內灣


In [53]:
# 輸入資料
ID = "J123073171"
FROM = "1000-臺北"
TO = "1210-新竹"
DATE = "2023/05/31"

# 00:00    08:00    16:00
#   |   or   |   or   |
# 08:00    16:00    23:59
# this time is end time
TIME = "23:59"


chrome.find_element(By.XPATH, "//*[@id='queryForm']/div[1]/div[3]/div[2]/label[2]").click()
for label in chrome.find_elements(By.XPATH, "//*[@id='queryForm']/div[2]/div[4]/div/div[2]/label"):
    label.click()
id_input = chrome.find_element(By.ID, "pid")
id_input.clear()
id_input.send_keys(ID)
from_input = chrome.find_element(By.ID, "startStation1")
from_input.clear()
from_input.send_keys(FROM)
to_input = chrome.find_element(By.ID, "endStation1")
to_input.clear()
to_input.send_keys(TO)
chrome.execute_script(f"document.getElementById('rideDate1').setAttribute('value', '{DATE}')")
Select(chrome.find_element(By.ID, "endTime1")).select_by_visible_text(TIME)

In [54]:
# 確認查詢
chrome.find_element(By.XPATH, "//*[@id='queryForm']/div[5]/input").click()

In [72]:
# 列出車次

class Car:
    def __init__(self, code: str, departure: str, arrival: str, time: str, price: str) -> None:
        self.code = code.replace(" ", "")
        self.departure = departure
        self.arrival = arrival
        self.time = time.replace(" ", "")
        self.price = price.replace(" ", "")

car_list = []
for tr in chrome.find_elements(By.CSS_SELECTOR, "tr[class='trip-column']"):
    tds = tr.find_elements(By.CSS_SELECTOR, "td")
    car_list.append(Car(tds[0].text, tds[2].text, tds[3].text, tds[4].text, tds[6].text))

for i in range(len(car_list)):
    car = car_list[i]
    print(f"{i + 1:2d} {car.code:>6} {car.departure} {car.arrival} {car.time} {car.price}")


 1  自強175 16:00 17:08 01小時08分 $177
 2  莒光521 16:10 17:39 01小時29分 $137
 3  自強139 17:00 18:08 01小時08分 $177
 4  自強141 17:30 18:39 01小時09分 $177
 5  自強177 17:40 19:08 01小時28分 $177
